In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [2]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[ 1.15628982e+00,  1.22897255e+00,  1.29172933e+00, ...,
          1.04420614e+00,  1.07510138e+00,  1.11135578e+00],
        [ 1.16081870e+00,  1.17503762e+00,  1.24922192e+00, ...,
          1.03911686e+00,  1.07712674e+00,  1.13184476e+00],
        [ 1.14556277e+00,  1.17094231e+00,  1.25146103e+00, ...,
          9.37992394e-01,  9.91703510e-01,  1.08357835e+00],
        ...,
        [-9.00610462e-02, -1.00929461e-01, -1.11157805e-01, ...,
         -5.45751490e-02, -6.68409243e-02, -7.86779597e-02],
        [-3.52831148e-02, -4.09698077e-02, -4.65110056e-02, ...,
         -1.71625074e-02, -2.34025400e-02, -2.94156615e-02],
        [ 2.72644330e-02,  2.61789840e-02,  2.50781476e-02, ...,
          3.06763388e-02,  2.95457691e-02,  2.84063276e-02]],

       [[ 1.23076105e+00,  1.36482346e+00,  1.44497204e+00, ...,
          1.03365445e+00,  1.05988741e+00,  1.12149143e+00],
        [ 1.17858207e+00,  1.24314678e+00,  1.34511971e+00, ...,
          1.01751697e+00,  1.05934691e

In [3]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [4]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4810,)


In [5]:
AO_ds.to_netcdf('MPI-ESM1-2-LR-AOindex-NDJF-Daily-1980-2014.nc')

In [6]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(2.9352098, dtype=float32)
Coordinates:
    time     datetime64[ns] 2007-01-11T12:00:00

In [7]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [8]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-2.9985616, dtype=float32)
Coordinates:
    time     datetime64[ns] 2003-12-22T12:00:00

In [9]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [10]:
zLevData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/NDJF_New_zg_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [11]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [12]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4810, lat: 96, lon: 192)>
[88657920 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 83.0 84.86 86.72 88.57
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-09-11T14:13:27Z altered by CMOR: Reordered dimension...

In [13]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [14]:
#z145E40N

In [15]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [16]:
#z20E55N

In [17]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [18]:
#z75E55N

In [19]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [20]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [21]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [22]:
EUVal

<xarray.DataArray 'zg' (time: 4810)>
array([-0.40658011, -0.11500372, -0.14822787, ...,  0.55817477,
        0.18297262, -0.46108966])
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 144.4

In [23]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [24]:
EU_ds.to_netcdf('MPI-ESM1-2-LR-EUindex-NDJF-Daily-1980-2014.nc')